In [6]:
import os
import logging
import re
from typing import List, Tuple
from oras.provider import Registry

logger = logging.getLogger(__name__)
logging.basicConfig(level=20)

def get_oras_client():
    """
    Consistent method to get an oras client
    """
    user = os.environ.get("ORAS_USER")
    password = os.environ.get("ORAS_PASS")
    reg = Registry()
    if user and password:
        print("Found username and password for basic auth")
        reg.set_basic_auth(user, password)
    else:
        logger.warning("No ORAS_USER or ORAS_PASS defined, no auth.")
    return reg


def split_uri(uri: str) -> Tuple[str, str]:
    if not uri.startswith("oci-artifact://"):
        raise ValueError("expected to start with 'oci-artifact://'")
    oras_target = re.sub(r'^oci-artifact://', '', uri)
    if ':' not in oras_target:
        raise ValueError("expected to contain at least a `:tag`")
    parts: List[str] = oras_target.split(':', 1)
    return oras_target, parts[1]


client = get_oras_client()
kserve_uri = "oci-artifact://quay.io/mmortari/demo20240606-orascsi-ociartifactrepo:latest"
oras_target, tag = split_uri(kserve_uri)
res = client.pull(target=oras_target, outdir="./tmp/")

INFO:oras.logger:Successfully pulled /Users/mmortari/git/demo20240430-orasRef/tmp/mnist.onnx.
